**Оглавление:**
1. [Изучение данных](#step1)  
2. [Предобработка данных](#step2) 
3. [Исследование заёмщиков](#step3)
4. [Общий вывод](#step4)


<a id="step1"></a>
## Изучение данных

Импортируем необходимые библиотеки:

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

Прочитаем файл с данными:

In [2]:
df = pd.read_csv('datasets/data.csv')

Получим первые 10 строк таблицы и общую информацию о данных:

In [3]:
display(df.head(10), df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


None

В таблице имеем 21525 строк и 12 столбцов. В 2 столбцах тип данных `float`, в 5 — `int`, и еще в 5  — `object`.

Количество значений в столбцах `days_employed` и `total_income` отличается от количества значений в других столбцах, значит, в этих стобцах есть пропущенные значения.

В столбце `days_employed` встречаются данные с отрицательным количеством дней трудового стажа.

В столбце `education` данные отличаются по регистру.

В таблице встречаются столбцы с категоризированными данными (пары `education`-`education_id` и  `family_status`-`family_status_id`), которые при необходимости можно выделить в отдельные словари.

В столбце `purpose` данные представлены в произвольном виде, например, расходы на свадьбу отражены как "сыграть свадьбу" и "на проведение свадьбы", а операции с недвижимостью как "покупка жилья", "операции с жильем" или "покупка жилья для семьи".

Отметим, что наиболее важными для нашего исследования являются следующие столбцы: `debt`, `children`, `family_status`/`family_status_id`, `total_income` и `purpose`. На корректности данных в них следует обратить особое внимание.

**Вывод**

В каждой строке таблицы представлены данные о потенциальном заёмщике (его пол, возраст, семейное положение и наличие/отсутствие детей, уровень образования и ежемесячного дохода, тип занятости и трудовой стаж, наличие опыта задолженности по кредиту и цель получения кредита).

Предварительно можно утверждать, что данных достаточно для ответа на поставленные заказчиком вопросы. Но в данных встречаются пропуски и артефакты, некоторые данные заполнены без учета регистра и не приведены к единому образцу. Также таблица загромождена категоризированными данными.

Для дальнейшей работы потребуется предобработка данных.

<a id="step2"></a>
## Предобработка данных

### Обработка пропусков

Посчитаем, сколько в таблице пропущенных значений:

In [4]:
display(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Мы имеем одинаковое количество пропущенных значений в столбцах `days_employed` и `total_income`. 

Судя по тому, что количество пропусков в обеих колонках одинаковое, можем предположить, что пропуски имеются в одних и тех же строках. Проверим это предположение, отобразив таблицу с пропущенными значениями:

In [5]:
display(df[df.isna().any(1)])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Предположение оказалось верным: мы имеем 2174 строки с пропусками в столбцах `days_employed` и `total_income`. Очевидной зависимости между этими и другими столбцами не наблюдается. Возможно, данные в этих столбцах являются рассчетными, и утеря/порча первичных данных, из которых рассчитываются общий трудовой стаж в днях и ежемесячный доход, повлекла за собой пропуски в этих столбцах. В учебном проекте восстановить эти данные мы не можем.

Данные из столбца `days_employed` не будут влиять на результаты исследования, поэтому смело можем заполнить пропуски нулями (не используем `'unknown'`, т.к. дальше будем меня формат в этом столбце). 

Применим для этого метод `fillna()`:

In [6]:
df['days_employed'] = df['days_employed'].fillna(0)

Для дальнейшего исследования нам важны значения в столбце `total_income`. Ежемесячный доход - количественная переменная, поэтому для оценки типичного значения выборки можно использовать среднее арифметическое или медиану. Поскольку мы имеем дело с показателем ежемесячного дохода, на мой взгляд, корректней взять медианное значение.

Предположим, что медианные значения дохода сильно различаются для разных типов занятости (возможно, пропуски нужно заполнить с учетом типа занятости). Выведем медианное значение дохода для разных типов занятости, а также посмотрим, какое количество данных у нас имеется по каждой из категорий:

In [7]:
display(df.groupby('income_type')['total_income'].median())
display(df.groupby('income_type')['total_income'].count())

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

income_type
безработный            2
в декрете              1
госслужащий         1312
компаньон           4577
пенсионер           3443
предприниматель        1
сотрудник          10014
студент                1
Name: total_income, dtype: int64

Видим, что медианное значение сильно отличается для разных категорий. Для корректного заполнения пропусков в столбце `total_income` заполним их медианным значением в соответствии с типом занятости:

In [8]:
df['total_income'] = df.groupby('income_type')['total_income'].apply(lambda s: s.fillna(s.median()))

Еще раз подсчитаем, сколько в таблице пропущенных значений, чтобы убедиться, что мы устранили все пропуски:

In [9]:
display(df.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Мы обнаружили пропуски значений в двух колонках: `days_employed` и `total_income`. 
В столбце `days_employed` пропущенные значения мы заменили на ноль. В столбце `total_income` пропущенные значения заменены на медианное значение по этому столбцу.

### Замена типа данных

В таблице имеем 2 столбца (`total_income` и `days_employed`) с данными вещественного типа `float`. Веротяно, это рассчитанные величины, поэтому они имеют по 6 знаков после запятой, что не несет никакой дополнительной инофрмации, но сильно загромождает нашу таблицу с данными. Приведем данные в этих столбцах к целочисленному типу `int`: 

In [10]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


Проверим, что тип данных в колонке изменился:

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


**Вывод**

Мы заменили тип данных в столбцах `total_income` и `days_employed`, тем самым визуально разгрузив нашу таблицу.

### Обработка дубликатов

Посчитаем явные дубликаты в таблице:

In [12]:
df.duplicated().sum()

54

Проверим, что из себя представляют эти дубликаты:

In [13]:
df[df.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,0,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
4182,1,0,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,0,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
7808,0,0,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
8583,0,0,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование
9238,2,0,34,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для сдачи
9528,0,0,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью
9627,0,0,56,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции со своей недвижимостью
10462,0,0,62,среднее,1,женат / замужем,0,F,пенсионер,0,118514,покупка коммерческой недвижимости


Из таблицы видим, что в дубликаты попали все строки с заполенными выше пропусками в колонках `days_employed` и `total_income`. Предположим, что из-за потери данных эти строки считываются как дубликаты, хотя на самом деле это всё данные о разных людях. В таком случае удаление этих строк повлечет за собой потерю данных, чего нам не хотелось бы. Оставим эти строки для дальнейшего исследования. 

Проанализируем важный для нашего исследования столбец `children	` при помощи метода `value_counts()`:

In [14]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Слегка неправдоподобными выглядят семьи с количеством детей, равным 20 и -1. Возможно, при вводе данных произошла опечатка (но для опечатки это слишком много раз, и почему опечались столько раз именно в написании 1 и 2?), а может это просто мусорные данные, наверняка мы не знаем. Поскольку мы имеем дело с количественной переменной, заменим при помощи цикла `for` и условной конструкции `if` эти данные на медианное значение по столбцу, и проверим, что изменения призошли:

In [15]:
for i in df['children']:
    if i == 20 or i == -1:
        df['children'] = df['children'].replace(i, df['children'].median())

df['children'].value_counts()

0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**

Методом `duplicated().sum()` мы обнаружили дубликаты, но, проанализировав данные, приняли решение не удалять их.
Также методом `value_counts()` мы обнаружили неправдоподобные данные в столбце `children` и заменили их на медианное значение по этому столбцу.

Также на первом шаге мы отметили, что в столбце `education` данные отличаются по регистру. Но, поскольку эти данные не повлияют на дальнейшее исследование, приводить их к единому регистру мы не будем.

### Лемматизация

Составим список из уникальных значений в колонке `purpose`:

In [16]:
unique_purpose = df['purpose'].unique()
unique_purpose

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Применим лемматизацию, чтобы упорядочить данные в полученном списке `unique_purpose`. Импортируем библиотеку `pymystem3` с функцией лемматизации на русском языке, а также вызовем специальный контейнер `Counter` из модуля `collections`:

In [17]:
m = Mystem() 

Посчитаем, какие слова встречаются в списке целей получения кредита. Для этого применим цикл `for` для лемматизации списка методом `lemmatize()` и посчитаем при помощи `Counter`, сколько раз слова встречаются в списке:

In [18]:
lemmas = []

for j in unique_purpose:
    lemmas += m.lemmatize(j)
display(Counter(lemmas)) 


Counter({'покупка': 10,
         ' ': 59,
         'жилье': 7,
         '\n': 38,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 1,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'подержанный': 1,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1})

**Вывод**

В результате лемматизации столбца `purpose` можно выделить 5 ключевых слов: `свадьба`, `образование`, `автомобиль`, `недвижимость` и `жилье`. Последние 2 можно будет объединить в одну категорию. Эти данные нам понадобятся далее для проведения категоризации.

### Категоризация данных

Для оценки зависимости между уровнем дохода и возвратом кредита в срок нам необходимо разделить значения в колонке ежемесячного дохода `total_income` на несколько категорий. Для этого используем метод ` pd.qcut()` и получим на выходе 4 интервала (q=4):

In [19]:
pd.qcut(df['total_income'], q=4)

0        (195549.0, 2265604.0]
1         (107798.0, 142594.0]
2         (142594.0, 195549.0]
3        (195549.0, 2265604.0]
4         (142594.0, 195549.0]
                 ...          
21520    (195549.0, 2265604.0]
21521     (142594.0, 195549.0]
21522    (20666.999, 107798.0]
21523    (195549.0, 2265604.0]
21524    (20666.999, 107798.0]
Name: total_income, Length: 21525, dtype: category
Categories (4, interval[float64, right]): [(20666.999, 107798.0] < (107798.0, 142594.0] < (142594.0, 195549.0] < (195549.0, 2265604.0]]

Поделим значения в этом столбце на 4 категории:
* до 107798;
* от 107799 до 145017;
* от 145018 до 195543;
* от 195544.

Напишем функцию `income_group`, которая в качестве аргумента принимает значение ежемесячного дохода и возвращает номер категории. Используя метод `apply`, применим функцию к столбцу `total_income` и занесем данные в новый столбец df `total_income_group`. Проверим, что функция отработала верно.

In [20]:
def income_group(income):
    if income <= 107798:
        return 0
    elif 107798 < income <= 145017:
        return 1
    elif 145017 < income <= 195543:
        return 2
    return 3

df['total_income_group'] = df['total_income'].apply(income_group)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_group
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,3
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,2
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,2
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,3
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,3
7,0,-152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,1
8,2,-6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,0
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,1


В разделе 2.1 мы выделили слова, которые определяют категории целей получения кредита.

Напишем функцию `purpose_group`, которая в качестве аргумента принимает строку, в которой указана цель получения кредита, и возвращает номер категории. Используя метод `apply`, применим функцию к столбцу `purpose` и занесем данные в новый столбец df `total_purpose_group`. Проверим, что функция отработала верно.

In [21]:
def purpose_group(target):
    if 'свадьб' in target:
        return 0
    elif 'автомоб' in target:
        return 1
    elif 'образов' in target:
        return 2
    elif 'недвижим' or 'жиль' in target:
        return 3

df['total_purpose_group'] = df['purpose'].apply(purpose_group)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_group,total_purpose_group
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,3,3
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1,1
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,2,3
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3,2
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,2,0
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,3,3
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,3,3
7,0,-152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,1,2
8,2,-6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,0,0
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,1,3


**Вывод**

При помощи функций мы разделили на категории значения в столбцах `total_income` и `purpose` (по 4 категории на каждый столбец).

<a id="step3"></a>
## Исследование заёмщиков

### Зависимость между наличием детей и возвратом кредита в срок

Для выведения результатов создадим новый df `df_children_pivot`.

Воспользуемся методом `pivot_table` со следующими аргументами:
- index='children' – группируем данные по количеству детей, столбец `children`;
- columns='debt' – значения, по которым происходит группировка, столбец `debt`;
- values='purpose' – используем любой из столбцов df (в данном случае `purpose`), т.к. далее применяем `count`;
- aggfunc='count' – применяемая функция `count`;
- margins=True – получаем обобщенные значения по каждой группе.

In [22]:
df_children_pivot = df.pivot_table(index='children', columns='debt', values='purpose', aggfunc='count', margins=True)
df_children_pivot

debt,0,1,All
children,,,
0,13200.0,1072.0,14272
1,4374.0,444.0,4818
2,1861.0,194.0,2055
3,303.0,27.0,330
4,37.0,4.0,41
5,9.0,NaN,9
All,19784.0,1741.0,21525


Добавим новый столбец `debt_rate`, в котором рассчитаем процент задолженностей в каждой группе, и выведем итоговую таблицу на экран:

In [23]:
df_children_pivot['debt_rate'] = df_children_pivot[1]/df_children_pivot['All']*100
df_children_pivot

debt,0,1,All,debt_rate
children,,,,
0,13200.0,1072.0,14272,7.511211
1,4374.0,444.0,4818,9.215442
2,1861.0,194.0,2055,9.440389
3,303.0,27.0,330,8.181818
4,37.0,4.0,41,9.756098
5,9.0,NaN,9,NaN
All,19784.0,1741.0,21525,8.088269


**Вывод**

В результате мы увидели, что процент задолженностей незначительно, но все же увеличивается при наличии у клиента детей. 

Клиенты без детей являются должниками в 7.5% случаев. Далее процент должников растет при наличии 1 (9.2%) и 2 (9.4%) детей. Среди клиентов с тремя детьми должниками являются 8.2%, с четырьмя детьми - 9.8%. Среди клиентов с 5 детьми (таких в нашей выборке всего 9 человек) должники отсутствуют.

### Зависимость между семейным положением и возвратом кредита в срок

Сначала посмотрим, на какие группы по семейному положению разбиты наши данные. Для этого сгруппируем df по столбцу `family_status_id` и выведем уникальные значения по столбцу `family_status`:

In [24]:
df.groupby('family_status_id')['family_status'].unique()

family_status_id
0          [женат / замужем]
1         [гражданский брак]
2           [вдовец / вдова]
3                [в разводе]
4    [Не женат / не замужем]
Name: family_status, dtype: object

Для выведения результатов создадим новый df `df_fam_status_pivot`.

Воспользуемся методом `pivot_table` со следующими аргументами:
- index= ['family_status_id','family_status'] – группируем данные по id и семейному статусу, столбцы `family_status_id` и `family_status`;
- columns='debt' – значения, по которым происходит группировка, столбец `debt`;
- values='purpose' – используем любой из столбцов df (в данном случае `purpose`), т.к. далее применяем `count`;
- aggfunc='count' – применяемая функция `count`;
- margins=True – получаем обобщенные значения по каждой группе.

In [25]:
df_fam_status_pivot = df.pivot_table(
    index=['family_status_id','family_status']
    , columns='debt', values='purpose', aggfunc='count', margins=True)

df_fam_status_pivot

,debt,0,1,All
family_status_id,family_status,,,
0,женат / замужем,11449,931,12380
1,гражданский брак,3789,388,4177
2,вдовец / вдова,897,63,960
3,в разводе,1110,85,1195
4,Не женат / не замужем,2539,274,2813
All,,19784,1741,21525


Добавим новый столбец `debt_rate`, в котором рассчитаем процент задолженностей в каждой группе, и выведем итоговую таблицу на экран:

In [26]:
df_fam_status_pivot['debt_rate'] = df_fam_status_pivot[1]/df_fam_status_pivot['All']*100
df_fam_status_pivot

,debt,0,1,All,debt_rate
family_status_id,family_status,,,,
0,женат / замужем,11449,931,12380,7.520194
1,гражданский брак,3789,388,4177,9.288963
2,вдовец / вдова,897,63,960,6.562500
3,в разводе,1110,85,1195,7.112971
4,Не женат / не замужем,2539,274,2813,9.740491
All,,19784,1741,21525,8.088269


**Вывод**

В результате мы увидели, что число задолженностей незначительно, но все же меняется в зависимости от семейного статуса клиента.

Чаще имеют задолженности по кредитам клиенты с семейным статусом `гражданский брак` (задолженность в 9.3% случаев)  и `Не женат / не замужем` (задолженность в 9.7% случаев). Немного реже задолженности по кредитам имеют клиенты с семейным статусом `женат / замужем` (задолженность в 7.5% случаев)  и `в разводе` (задолженность в 7.1% случаев).  Реже всего имеют задолженность клиенты со статусом `вдовец / вдова` (задолженность в 6.6% случаев). 

### Зависимость между уровнем дохода и возвратом кредита в срок

Ранее мы разделили доходы клиентов на 4 категории.

Для выведения результатов создадим новый df `df_income_pivot`.

Воспользуемся методом `pivot_table` со следующими аргументами:
- index='total_income_group' – группируем данные по ранее выделенным категориям дохода, столбец `total_income_group`;
- columns='debt' – значения, по которым происходит группировка, столбец `debt`;
- values='purpose' – используем любой из столбцов df (в данном случае `purpose`), т.к. далее применяем `count`;
- aggfunc='count' – применяемая функция `count`;
- margins=True – получаем обобщенные значения по каждой группе.

In [27]:
df_income_pivot = df.pivot_table(index='total_income_group', columns='debt', values='purpose', aggfunc='count', margins=True)
df_income_pivot

debt,0,1,All
total_income_group,,,
0,4955,427,5382
1,5302,510,5812
2,4531,418,4949
3,4996,386,5382
All,19784,1741,21525


Добавим новый столбец `debt_rate`, в котором рассчитаем процент задолженностей в каждой группе, и выведем итоговую таблицу на экран:

In [28]:
df_income_pivot['debt_rate'] = df_income_pivot[1]/df_income_pivot['All']*100
df_income_pivot

debt,0,1,All,debt_rate
total_income_group,,,,
0,4955,427,5382,7.933854
1,5302,510,5812,8.774948
2,4531,418,4949,8.446151
3,4996,386,5382,7.172055
All,19784,1741,21525,8.088269


**Вывод**

В результате мы увидели, что число задолженностей незначительно, но все же меняется в зависимости от ежемесячного дохода клиента. 

Чаще всего имеют задолженности по кредитам клиенты с ежемесячным доходом от 107799 до 145017 - задолженность в 8.8% случаев. Реже всего задолженности по кредитам у клиентов с ежемесячным доходом более 195544 - задолженность в 7.2% случаев. Чуть чаще имеют задолженность клиенты с ежемесячным доходом менее 107798 (задолженность в 7.9% случаев) и клиенты ежемесячным доходом от 145018 до 195543 (задолженность в 8.4% случаев).

### Зависимость цели кредита на его возврат в срок

Ранее мы разделили цели получения кредита на 4 категории.

Для выведения результатов создадим новый df `df_purpose_pivot`.

Воспользуемся методом `pivot_table` со следующими аргументами:
- index='total_purpose_group' – группируем данные по ранее выделенным категориям целей получения кредита, столбец `total_purpose_group`;
- columns='debt' – значения, по которым происходит группировка, столбец `debt`;
- values='purpose' – используем любой из столбцов df (в данном случае `purpose`), т.к. далее применяем `count`;
- aggfunc='count' – применяемая функция `count`;
- margins=True – получаем обобщенные значения по каждой группе.

In [29]:
df_purpose_pivot = df.pivot_table(index='total_purpose_group', columns='debt', values='purpose', aggfunc='count', margins=True)
df_purpose_pivot

debt,0,1,All
total_purpose_group,,,
0,2162,186,2348
1,3912,403,4315
2,3652,370,4022
3,10058,782,10840
All,19784,1741,21525


Добавим новый столбец `debt_rate`, в котором рассчитаем процент задолженностей в каждой группе, и выведем итоговую таблицу на экран:

In [30]:
df_purpose_pivot['debt_rate'] = df_income_pivot[1]/df_income_pivot['All']*100
df_purpose_pivot

debt,0,1,All,debt_rate
total_purpose_group,,,,
0,2162,186,2348,7.933854
1,3912,403,4315,8.774948
2,3652,370,4022,8.446151
3,10058,782,10840,7.172055
All,19784,1741,21525,8.088269


**Вывод**

В результате мы увидели, что число задолженностей незначительно, но все же меняется в зависимости от цели кредита. 

Чаще всего имеют задолженности по кредитам клиенты, цели кредита которых относятся к категориям `автомобиль` или `образование` - задолженность в 8.8% и 8.4% случаев соответственно. Реже задолженности по кредитам имеют клиенты, цели кредита которых относятся к категориям `свадьба` (задолженность в 7.9% случаев) или `недвижимость/жильё` (задолженность в 7.2% случаев).

<a id="step4"></a>
## Общий вывод

В первую очередь хотелось бы отметить, что по любой из выделенных нами категорий процент задолженностей составлял менее 10% (что, на мой взгляд, довольно неплохой показатель).

Также хочется отметить, что разница между показателями процента задолженности в каждой из катерогий очень незначительная и не превышает максимум 3% по категории.

Расположим клиентов в порядке возрастания вероятности задолженности по кредиту (т.е. чем больше число, тем больше вероятность, что клиент будет должником) в разных категориях:

* Наличие/отсутствие детей:
1. 5 детей.
2. Дети отсутствуют.
3. 3 детей.
4. 1 ребенок.
5. 2 детей.
6. 4 детей.

* Семейное положение:
1. Вдовец / вдова.
2. В разводе.
3. Женат / замужем.
4. Гражданский брак.
5. Не женат / не замужем.

* Ежемесячный доход:
1. Более 195544.
2. Менее 107798.
3. От 145018 до 195543.
4. От 107799 до 145017.

* Цель кредита:
1. Свадьба.
2. Недвижимость/жильё.
3. Образование.
4. Автомобиль.
